In [28]:
import numpy as np
import xarray as xr
import gsw

In [2]:
# Specify geometry of vertical grid
# (based on MOM6 input parameters)

NK = 20
MAXIMUM_DEPTH = 4000
MINIMUM_DEPTH = 0

# Resolve vertical grid
# Uniform
Z = np.linspace(MINIMUM_DEPTH,MAXIMUM_DEPTH,NK)

# Specify geometry of horizontal grid
# (for extension of profile to 3D grid)

# Latitude and longitude
SOUTHLAT=-60.0
LENLAT=30.0
WESTLON=0.0
LENLON=100.0

# Number of grid cells
NI=100
NJ=20

# Grid point positions (tracer point)
X=np.linspace(WESTLON,WESTLON+LENLON,NI+1)
X1d=(X[1:] + X[:-1]) / 2
Y=np.linspace(SOUTHLAT,SOUTHLAT+LENLAT,NJ+1)
Y1d=(Y[1:] + Y[:-1]) / 2

# Array of grid point positions
X,Y=np.meshgrid(X1d,Y1d)

In [5]:
# Load T,S initialization file that will be used to set values in sponge layer
rootdir = '/work/gam/MOM6/initialization/'
config = 'channel'
filename = 'ts_analytic.nc'
TS = xr.open_dataset(rootdir+config+'/'+filename,decode_times=False)

In [42]:
# Set inverse damping rate across model domain (in s-1)
idampval = 1/(1*86400)
ydamp = 2; # Degrees of lat to damp over

idamp = xr.DataArray(np.zeros([NI,NJ]),coords=[X1d,Y1d],dims=['lon','lat'])
sponge_region = idamp.coords['lat']>SOUTHLAT+LENLAT-ydamp
idamp=xr.where(sponge_region,idampval,idamp)
print(idamp)

<xarray.DataArray (lat: 20, lon: 100)>
array([[0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.157407e-05, 1.157407e-05, 1.157407e-05, ..., 1.157407e-05,
        1.157407e-05, 1.157407e-05]])
Coordinates:
  * lat      (lat) float64 -59.25 -57.75 -56.25 -54.75 ... -33.75 -32.25 -30.75
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 5.5 ... 95.5 96.5 97.5 98.5 99.5


In [47]:
# Set 'interface' height (just set to ALE coordinate z-grid)
# Broadcast depth across lat-lon domain
eta = xr.DataArray(-Z,coords=[Z],dims=['depth'])
eta = eta*xr.DataArray(np.ones([NI,NJ]),coords=[X1d,Y1d],dims=['lon','lat'])

<xarray.DataArray (depth: 20, lon: 100, lat: 20)>
array([[[   -0.      ,    -0.      , ...,    -0.      ,    -0.      ],
        [   -0.      ,    -0.      , ...,    -0.      ,    -0.      ],
        ...,
        [   -0.      ,    -0.      , ...,    -0.      ,    -0.      ],
        [   -0.      ,    -0.      , ...,    -0.      ,    -0.      ]],

       [[ -210.526316,  -210.526316, ...,  -210.526316,  -210.526316],
        [ -210.526316,  -210.526316, ...,  -210.526316,  -210.526316],
        ...,
        [ -210.526316,  -210.526316, ...,  -210.526316,  -210.526316],
        [ -210.526316,  -210.526316, ...,  -210.526316,  -210.526316]],

       ...,

       [[-3789.473684, -3789.473684, ..., -3789.473684, -3789.473684],
        [-3789.473684, -3789.473684, ..., -3789.473684, -3789.473684],
        ...,
        [-3789.473684, -3789.473684, ..., -3789.473684, -3789.473684],
        [-3789.473684, -3789.473684, ..., -3789.473684, -3789.473684]],

       [[-4000.      , -4000.      , ...

In [49]:
idamp.name='idamp'
eta.name='eta'
sponge = xr.merge([TS.T_interp_3D,TS.S_interp_3D,idamp,eta])
# Transpose
sponge = sponge.transpose('depth','lat','lon')

rootdir = '/work/gam/MOM6/forcing/'
config = 'channel'
filename = 'sponge.nc'
sponge.to_netcdf(rootdir+config+'/'+filename)